<a href="https://colab.research.google.com/github/mparag019/DL-Assignment-2/blob/main/PART-B/DL_Assign_2_PART_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'dataset2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4749322%2F8053052%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240407%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240407T142307Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dec75f514e87132b001ad4d147702ca2b48cc20c2703825e397a9e08fd7bd388f13b42a86ec657715b603154b9954b28075a5d8636f56c31d87eb1e5834d2d6cf8c5b5cf7fe41ca6a56be59b2cadf934cad1a2e09db6b4e6dee9ce7e3ae13281e93e07fceb72dd9a5b97ac020229f09935bfcdef7bea78a707c61fb7ee1ed002e3fb5775854d04628d0ab6c4e96fb2f431113df0e3477471973f97b43c9dbe8c6fb00f7452317f6077bfdc572ef15ca3d4f10353fc36d47dce073c7a52cc4051638417e4dae1027646ce5f5a19d7836d21e891bd94d651b7f8b9dbb452ea115bc0ba5b54947bb54417b14b60257369072e8aedf98596f016a7a4a63de1ff3a530'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler, random_split, ConcatDataset
from torchvision.datasets import ImageFolder
import wandb
from wandb.sdk.wandb_run import Run
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import numpy as np



# Check if CUDA (GPU) is available, and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Set up data transformations
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# Load the dataset
train_data = ImageFolder('/kaggle/input/dataset1/inaturalist_12K/train', transform=train_transforms)
test_data = ImageFolder('/kaggle/input/dataset1/inaturalist_12K/val', transform=test_transforms)


In [ ]:
# Count the number of samples in each class
class_counts = {}
pbar = tqdm(total=len(train_data))
for _, label in train_data:
    if label not in class_counts:
        class_counts[label] = 0
    class_counts[label] += 1
    pbar.set_postfix()
    pbar.update(1)

pbar.close()

# Calculate the number of samples per class for validation set
val_size_per_class = {label: int(count * 0.2) for label, count in class_counts.items()}

# Initialize lists to hold indices for train and validation sets
train_indices = []
val_indices = []

# Iterate through the dataset and assign samples to train or validation set
pbar = tqdm(total=len(train_data))
for idx, (_, label) in enumerate(train_data):
    if val_size_per_class[label] > 0:
        val_indices.append(idx)
        val_size_per_class[label] -= 1
    else:
        train_indices.append(idx)
    pbar.set_postfix()
    pbar.update(1)

pbar.close()

# Create SubsetRandomSampler for train and validation sets
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)


In [ ]:
# Function to calculate accuracy
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    accuracy = correct / labels.size(0)
    return accuracy

In [ ]:
# Training loop
def training_model(epochs, optimizer, criterion, model, train_loader, val_loader):
    for epoch in range(epochs):
        model.train()
        training_loss = 0.0
        train_accuracy = 0.0
        pbar = tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{epochs}')
        for images, labels in train_loader:
            optimizer.zero_grad()
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            training_loss += loss.item()
            train_accuracy += calculate_accuracy(outputs, labels)
            pbar.set_postfix({'Train Loss': training_loss / (pbar.n + 1), 'Train Acc': train_accuracy / (pbar.n + 1)})
            pbar.update(1)

        pbar.close()


        model.eval()
        val_loss = 0.0
        val_accuracy = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_loss += criterion(outputs, labels).item()
                val_accuracy += calculate_accuracy(outputs, labels)

        train_accuracy /= len(train_loader)
        training_loss /= len(train_loader)
        val_loss /= len(val_loader)
        val_accuracy /= len(val_loader)
        print(f'Epoch {epoch+1}/{epochs}, Train_Loss: {training_loss:.4f},  Train_Acc: {train_accuracy:.4f},  Val_Loss: {val_loss:.4f},  Val_Accuracy: {val_accuracy:.4f}')
        wandb.log({"epoch": epoch+1, "train_loss": training_loss, "val_loss": val_loss, "val_accuracy": val_accuracy, "train_accuracy": train_accuracy})
    return model



In [ ]:
sweep_config = {
    'method': 'bayes',  # Random search method
    'metric': {'goal': 'maximize', 'name': 'val_accuracy'},  # Metric to optimize
    'parameters': {
        'epochs': {'values':[5, 10]},
        'batch_size': {'values':[32, 64]},
        'num_filters': {'values': [32, 64, 128]},
        'activation': {'values': ['ReLU', 'GELU', 'SiLU', 'Mish']},
        'filter_organization': {'values': ['same', 'double', 'halve']},
        'data_augmentation': {'values': [True, False]},
        'batch_norm': {'values': [True, False]},
        'dropout': {'values': [0.2, 0.3]},
        'strategy':{'values':[1, 2, 3]}
    }
}

In [ ]:
def apply_additional_transforms(loader, additional_transforms, batch_size):
    transformed_dataset = []
    original_dataset = []
    pbar = tqdm(total=len(loader))
    for images, labels in loader:
        images1 = additional_transforms(images)
        for i in range(batch_size):
            original_dataset.append((images[i], labels[i]))
            transformed_dataset.append((images1[i], labels[i]))
        pbar.set_postfix()
        pbar.update(1)

    pbar.close()
    return original_dataset, transformed_dataset



In [ ]:
def augment_data(data_augmentation, train_loader, batch_size):
    if data_augmentation:
        additional_transforms = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
            transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        ])

        # Apply additional transformations to the new DataLoader
        original_dataset, transformed_dataset = apply_additional_transforms(train_loader, additional_transforms, batch_size)
        combined_dataset = ConcatDataset([original_dataset, transformed_dataset])

        # Create a new DataLoader using the combined dataset
        combined_loader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=True)
    else:
        combined_loader = train_loader
    return combined_loader

In [ ]:
def train_CNN(num_filters, activation, filter_organization, data_augmentation, batch_norm, dropout, batch_size, epochs):

    # Load pre-trained model (ResNet50)
    model = torchvision.models.resnet50(pretrained=True)
    model.to(device)
    if strategy == 1:
    # Strategy 1: Freeze all layers except the last layer
        for param in model.parameters():
            param.requires_grad = False

        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 101)  # 101 classes in iNaturalist
    elif strategy == 2:
    # Strategy 2: Freeze layers up to a certain depth
    # Freeze layers up to layer 4
        for name, param in model.named_parameters():
            if 'layer4' not in name:  # Freeze layers up to layer 4
                param.requires_grad = False
    else:
    # Strategy 3: Layer-wise fine-tuning
        for param in model.layer4.parameters():
            param.requires_grad = True

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Create DataLoader instances for train and validation sets using the samplers
    train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
    val_loader = DataLoader(train_data, batch_size=batch_size, sampler=val_sampler)

    combined_loader = augment_data(data_augmentation, train_loader, batch_size)

    model = training_model(epochs, optimizer, criterion, model, combined_loader, val_loader)
    return model

In [ ]:
wandb.login(key = "1d2c93cf7ddd48a63114848b66796301171827b6")
sweep_id = wandb.sweep(sweep_config, project='DL-Assignment-2')

# Define your training function
def train():

    # Initialize Wandb run with custom run name
    with wandb.init() as run:

        # Use wandb.config to access hyperparameters in your training script
        config = wandb.config
        num_filters = config['num_filters']
        activation = config['activation']
        filter_organization = config['filter_organization']
        data_augmentation = config['data_augmentation']
        batch_norm = config['batch_norm']
        batch_size = config['batch_size']
        epochs = config['epochs']
        dropout = config['dropout']
        # Generate a custom run name based on hyperparameters
        run_name = "Part-B_" + "epochs_" + str(epochs) + "_nFilters_" + str(num_filters) + "_activation_" + str(activation)+ "_filterOrg_" + str(filter_organization) + "_batchSize_" + str(batch_size)
        wandb.run.name = run_name

        model = train_CNN(num_filters, activation, filter_organization, data_augmentation, batch_norm, dropout, batch_size, epochs)

        # Test the model
        test_loader = DataLoader(test_data, batch_size=batch_size)
        model.eval()

        test_accuracy = 0.0
        pbar = tqdm(total=len(test_loader))
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                images.to("cpu")
                labels.to("cpu")
                for i in range(len(images)):
                    image = images[i]
                    label = labels[i]
                    output = outputs[i].argmax(dim = 0)
                    if (label == output):
                        test_accuracy += 1
                pbar.set_postfix()
                pbar.update(1)

        pbar.close()
        wandb.login(key = "1d2c93cf7ddd48a63114848b66796301171827b6")
        with wandb.init( project='DL-Assignment-2') as run:
            run_name = "test_accuracy - Part B"
            wandb.run.name = run_name
            test_accuracy /= len(test_data)
            print(test_accuracy)
            wandb.log({"test_accuracy": test_accuracy})

        wandb.finish()

# Run the sweep
wandb.agent(sweep_id, function=train, count=20)
wandb.finish()